# Loading the model 

Before we begin with the detection, we must load the model that we had saved in the previous process using the load_model() by including the path of the .model file.

In [1]:
!pip install tensorflow

In [2]:
import tensorflow as tf
tf.__version__

'2.8.0'

In [3]:
from tensorflow.keras.models import load_model
detector = load_model(r'dummy.model')

# Starting the stream for Mask Detection in Real Time 

To detect the mask in a live stream we will follow the following steps:
1. start the video stream 
2. Capture the frame from the stream
3. Resize the frame
4. Detect faces in the frame using haarcascade classifier
5. Get the predictions using the saved model
6. Depending on the results draw rectangle and put text on the faces accordingly

The following are some of the important functions that we will use for our process.<br>
img_to_array() - Converts the image to a numpy array<br>
detectmultiscale() - Detects objects in the image<br>
tf.expand_dims() - Inserts a dimension of length 1 and returns a tensor<br>
tf.nn.softmax() - used for computing softmax activations<br>
numpy.argmax() - returns the indices of the values that are maximum along the x-axis<br>
    
<strong>Alternatively, you can choose other face detection technique instead of haarcascade, since it is the most basic technique to detect faces. And sometimes the results are not very efficient. 
You can use the opencv caffe model for face detection for variation in your results

In [4]:
!pip install opencv-python

In [5]:
import tensorflow as tf
import cv2
import numpy

#starting the video stream
cap = cv2.VideoCapture(0) 

#using the XML file for haarcascade classifier
classifier = cv2.CascadeClassifier(r"haarcascade_frontalface_default.xml")


In [6]:
detector.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1_pad (ZeroPadding2D)      (None, 225, 225, 3)  0           ['input_1[0][0]']                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['Conv1_pad[0][0]']              
                                )                                                                 
                                                                               

 ization)                                                                                         
                                                                                                  
 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malizatio

                                                                                                  
 block_6_project_BN (BatchNorma  (None, 14, 14, 64)  256         ['block_6_project[0][0]']        
 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
          

 block_10_depthwise_BN (BatchNo  (None, 14, 14, 384)  1536       ['block_10_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_

 block_14_expand_relu (ReLU)    (None, 7, 7, 960)    0           ['block_14_expand_BN[0][0]']     
                                                                                                  
 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_

In [7]:
#using the loop to watch the stream in real time.
while True:
    (success, frame) = cap.read()  #reading the frame from the stream 
    new_image = cv2.resize(frame, (frame.shape[1] // 1, frame.shape[0] // 1)) #resizing the frame to speed up the process of detection
    face = classifier.detectMultiScale(new_image) #detecting faces from the frame(ROI)
    for x,y,w,h in face:
        try:
            face_img = new_image[y:x+h, x:x+w] #getting the coordinates for the face detected
            resized= cv2.resize(face_img,(224,224)) #resizing the  face detected to fit into the model in the shape(224,224)
            image_array = tf.keras.preprocessing.image.img_to_array(resized) #converting the detected image into an array 
            image_array = tf.expand_dims(image_array,0) #expanding the dimensions to fit in the model
            predictions = detector.predict(image_array) #making predictions on the ROI
            score = tf.nn.softmax(predictions[0]) #getting the results 
            label = numpy.argmax(score)
        except Exception as e:
            print('bad frame')
            
        if label == 0:
            cv2.rectangle(new_image,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.putText(new_image,"mask",(x,y),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,0), 2)
        elif label == 1:
            cv2.rectangle(new_image,(x,y),(x+w,y+h),(0,0,255),2)
            cv2.putText(new_image,'no_mask',(x,y),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,255), 2)
        else:
            None
    #displaying the window after predicting the outcome
    cv2.imshow('face_window', new_image)
    print(numpy.argmax(score), 100*numpy.max(score))
    #waitkey to terminate the loop
    key = cv2.waitKey(10) 
    if key == ord('q'):
        break
#release the stream 
cap.release()
cv2.destroyAllWindows()

1 0.27136169373989105
1 0.2713212510570884
1 0.27136169373989105
1 0.2713616006076336
1 0.2713612047955394
1 0.27136169373989105
1 0.27136169373989105
1 0.27136148419231176
1 0.27136020362377167
1 0.27136169373989105
1 0.27134467381983995
1 0.27136169373989105
1 0.27136169373989105
1 0.27132511604577303
1 0.2713410183787346
1 0.2713616006076336
1 0.2713615773245692
1 0.27136055286973715
bad frame
1 0.271361181512475
1 0.2713564084842801
1 0.27136169373989105
1 0.27136169373989105
1 0.2653111703693867
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27133619878441095
1 0.27136169373989105
1 0.27136169373989105
1 0.27136148419231176
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
bad frame
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105


0 0.2713616006076336
0 0.2713616006076336
0 0.2713616006076336
0 0.2713616006076336
0 0.2713616006076336
0 0.2713616006076336
0 0.2713616006076336
0 0.2713616006076336
0 0.2713616006076336
0 0.2713616006076336
0 0.2713616006076336
0 0.2713616006076336
0 0.2713616006076336
0 0.2713616006076336
0 0.2713616006076336
0 0.2713616006076336
0 0.2713616006076336
0 0.2713616006076336
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
0 0.26455202605575323
0 0.26455202605575323
0 0.26455202605575323
0 0.26455202605575323
0 0.26455202605575323
0 0.26455202605575323
0 0.26455202605575323
0 0.26455202605575323
0 0.26455202605575323
0 0.26455202605575323
0 0.26455202605575323
0 0.26455202605575323
0 0.26455202605575323
0 0.26455202605575323
0 0.26

0 0.2710262546315789
1 0.27136169373989105
0 0.19396161660552025
1 0.2712620422244072
1 0.27136029675602913
1 0.27136169373989105
1 0.27136169373989105
1 0.27136108838021755
1 0.25930905248969793
1 0.2713616471737623
1 0.2713616006076336
1 0.2713616006076336
1 0.2713616006076336
1 0.2713616006076336
1 0.2713616006076336
1 0.2713616006076336
1 0.2713616006076336
1 0.2713616006076336
1 0.2713616006076336
1 0.2713616006076336
1 0.2713616006076336
1 0.2713616006076336
1 0.2713616006076336
1 0.2713616006076336
1 0.2713616006076336
0 0.27133673429489136
0 0.27133673429489136
0 0.27133673429489136
0 0.27133673429489136
0 0.27133673429489136
0 0.27133673429489136
0 0.27133673429489136
bad frame
0 0.27133673429489136
0 0.27133673429489136
0 0.27133673429489136
0 0.27133673429489136
0 0.27133673429489136
0 0.27133673429489136
0 0.27133673429489136
bad frame
0 0.27133673429489136
0 0.27133673429489136
0 0.27133673429489136
0 0.27133673429489136
0 0.27133673429489136
0 0.27133673429489136
0 0.2713

0 0.27136169373989105
1 0.2713307039812207
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.2713612047955394
bad frame
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
bad frame
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
bad frame
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
bad frame
0 0.26480958331376314
1 0.27136169373989105
1 0.27136169373989105
0 0.22648475132882595
1 0.27133338153362274
1 0.27136169373989105
1 0.271361693739

1 0.27135892305523157
1 0.27136169373989105
0 0.265358155593276
0 0.265358155593276
0 0.265358155593276
0 0.265358155593276
1 0.27136169373989105
1 0.27136169373989105
0 0.18696733750402927
0 0.27136169373989105
0 0.27136155404150486
0 0.2713616006076336
0 0.2711374778300524
0 0.2705476013943553
0 0.2713252091780305
1 0.17871569143608212
0 0.27100201696157455
0 0.27136169373989105
0 0.21349440794438124
0 0.27113701216876507
1 0.17057325458154082
0 0.2713082358241081
0 0.2713082358241081
0 0.27136169373989105
0 0.27136169373989105
0 0.27136169373989105
0 0.27136169373989105
0 0.27136169373989105
0 0.27136169373989105
0 0.27136169373989105
0 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.27136169373989105
1 0.2709890017285943
1 0.2713373163715005
1 0.2713373163715005
1 0.2713373163715005
1 0.2713373163715005
1 0.2713373163715005
1 0.2713373163715005
1 0.2713373163715005
1 0.2713373163715005
1 0.2713373163715005
1 0.27133731